In [ ]:
# Importing Required Libraries
import torchvision
import torch
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy
import torch.nn as nn
import torch.optim as optim
from random import randint
import time
import glob
from torch.autograd import Variable

In [ ]:
# Selcting Device CPU or GPU 
device= torch.device("cuda")
# device= torch.device("cpu")
print(device)

cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sat Jan 22 06:22:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Changing Current directory to drive
%cd '/content/drive/My Drive'

/content/drive/My Drive


In [ ]:
# # Unziping the Dataset
# !unzip -uq Dataset.zip

In [ ]:
# Specifying the path of training and testing data
train_path='./Component2_data/train'
test_path='./Component2_data/test'

In [ ]:
# Getting the number of classes present in the dataset
classes=os.listdir('./Component2_data/test')
len_class=len(classes)
len_class

3

In [ ]:
# Defining train and test transform
train_transform=transforms.Compose([transforms.Resize((48,48)),transforms.RandomHorizontalFlip(),transforms.ToTensor()])
test_transform=transforms.Compose([transforms.Resize((48,48)),transforms.RandomHorizontalFlip(),transforms.ToTensor()])

In [ ]:
# Getting train and test data after applying transform
train_data=torchvision.datasets.ImageFolder(root=train_path,transform=train_transform)
test_data=torchvision.datasets.ImageFolder(root=test_path,transform=test_transform)

In [ ]:
# Coverting train_data to train_loader
train_load=torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True)
test_load=torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=False)

In [ ]:
# CNN class
class convnet(nn.Module):

    def __init__(self):

        super(convnet, self).__init__()

        # block 1:         3 x 48 x 48 --> 96 x 24 x 24        
        self.conv1a = nn.Conv2d(3,   96,  kernel_size=3, padding=1 )
        self.conv1b = nn.Conv2d(96,  96,  kernel_size=3, padding=1 )
        self.pool1  = nn.MaxPool2d(2,2)
        # self.batch1 = nn.BatchNorm2d(96)

        # block 2:         96 x 24 x 24  --> 192 x 12 x 12
        self.conv2a = nn.Conv2d(96,  192, kernel_size=3, padding=1 )
        self.conv2b = nn.Conv2d(192, 192, kernel_size=3, padding=1 )
        self.pool2  = nn.MaxPool2d(2,2)

        # block 3:         192 x 12 x 12 --> 384 x 6 x 6       
        self.conv3a = nn.Conv2d(192, 384, kernel_size=3, padding=1 )
        self.conv3b = nn.Conv2d(384, 384, kernel_size=3, padding=1 )
        self.pool3  = nn.MaxPool2d(2,2)
        
        #block 4:          384 x 6 x 6--> 768 3 x 3
        self.conv4a = nn.Conv2d(384, 384*2, kernel_size=3, padding=1 )
        self.pool4  = nn.MaxPool2d(2,2)

        # linear layers:   768 x 3 x 3 --> 768*3*3*2 --> 768*3*3*2 --> 768*3*3*2 --> 3
        self.linear1 = nn.Linear(768*3*3, 768*3*3*2)
        self.linear2 = nn.Linear(768*3*3*2,768*3*3*2)
        self.linear3 = nn.Linear(768*3*3*2, 3)

        self.dropout=nn.Dropout(0.5)

        # self.batch2 = nn.BatchNorm1d(768*3*3*2)

    def forward(self, x):

        # block 1:          3 x 48 x 48 --> 96 x 24 x 24 
        x = self.conv1a(x)
        x = torch.relu(x)
        x = self.conv1b(x)
        x = torch.relu(x)
        x = self.pool1(x)
        # x = self.batch1(x)

        # block 2:         96 x 24 x 24  --> 192 x 12 x 12
        x = self.conv2a(x)
        x = torch.relu(x)
        x = self.conv2b(x)
        x = torch.relu(x)
        x = self.pool2(x)

        # block 3:         192 x 12 x 12 --> 384 x 6 x 6 
        x = self.conv3a(x)
        x = torch.relu(x)
        x = self.conv3b(x)
        x = torch.relu(x)
        x = self.pool3(x)

        #block 4:          384 x 6 x 6--> 768 3 x 3
        x = self.conv4a(x)
        x = torch.relu(x)
        x = self.pool4(x)

        # linear layers:   768 x 3 x 3 --> 768*3*3*2 --> 768*3*3*2 --> 768*3*3*2 --> 3
        x = x.view(-1, 768*3*3)
        x = self.linear1(x)
        x = torch.relu(x)
        # x = self.batch2(x)
        x = self.linear2(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.linear3(x) 
        
        return x

In [ ]:
# Creating a model of CNN
net=convnet()

print(net)

# Loading model to device
net = net.to(device)

convnet(
  (conv1a): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1b): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2a): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2b): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3b): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=6912, out_features=13824, bias=True)
  (linear2): Linear(in_f

In [ ]:
# Initializing CrossEntropyLoss and values for the learning rate and Batch size
criterion = nn.CrossEntropyLoss()
my_lr=0.25
bs= 30

In [ ]:
# Getting count of the number of training and testing data
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)

321 99


In [ ]:
# Training the CNN model with training data and Evaluating the accuracy of the model
max_acc=[]

for epoch in range(1,50):
    
    # divide the learning rate by 2 at epoch 10, 14 and 18
    if epoch % 10==0 :
        my_lr = my_lr / 2
    
    net.train()
    
    # set the running quatities to zero at the beginning of the epoch
    train_acc=0.0
    train_loss=0.0
    
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )

    for i , (images,labels) in enumerate(train_load):

      if torch.cuda.is_available():
        images=Variable(images.cuda())
        labels=Variable(labels.cuda())
    
      # Set the gradients to zeros
      optimizer.zero_grad()
        
      output=net(images)
      loss=criterion(output,labels)
      loss.backward()
      optimizer.step()

      train_loss+=loss.cpu().data*images.size(0)
      _,prediction=torch.max(output.data,1)

      train_acc+=int(torch.sum(prediction==labels.data))
    
    train_acc=train_acc/train_count
    train_loss=train_loss/train_count
       

    net.eval()

    test_acc=0.0
 
    for i , (images,labels) in enumerate(test_load):

      if torch.cuda.is_available():
        images=Variable(images.cuda())
        labels=Variable(labels.cuda())
        
      output=net(images)

      _,prediction=torch.max(output.data,1)

      test_acc+=int(torch.sum(prediction==labels.data))

    test_acc=test_acc/test_count

    max_acc.append(test_acc)

    print('Epoch:'+str(epoch)+'  Train loss:'+str(train_loss)+'  Train Accuracy:'+str(train_acc)+ '  Test Accuracy:'+str(test_acc))

    if epoch%10 == 0:
      print("\nTest Accuracy after "+str(epoch)+" epoch "+str(max(max_acc))+"\n")

Epoch:1  Train loss:tensor(1.1011)  Train Accuracy:0.2897196261682243  Test Accuracy:0.35353535353535354
Epoch:2  Train loss:tensor(1.1093)  Train Accuracy:0.35202492211838005  Test Accuracy:0.30303030303030304
Epoch:3  Train loss:tensor(1.1075)  Train Accuracy:0.308411214953271  Test Accuracy:0.35353535353535354
Epoch:4  Train loss:tensor(1.1073)  Train Accuracy:0.35202492211838005  Test Accuracy:0.3434343434343434
Epoch:5  Train loss:tensor(1.1058)  Train Accuracy:0.30218068535825543  Test Accuracy:0.35353535353535354
Epoch:6  Train loss:tensor(1.1062)  Train Accuracy:0.32710280373831774  Test Accuracy:0.35353535353535354
Epoch:7  Train loss:tensor(1.1090)  Train Accuracy:0.35202492211838005  Test Accuracy:0.3434343434343434
Epoch:8  Train loss:tensor(1.1061)  Train Accuracy:0.3177570093457944  Test Accuracy:0.30303030303030304
Epoch:9  Train loss:tensor(1.1028)  Train Accuracy:0.3302180685358255  Test Accuracy:0.35353535353535354
Epoch:10  Train loss:tensor(1.1067)  Train Accuracy:0